<!-- ## SMALL-SCALE CROP-MAPPING(Food security) -->

<center><h1>Weather Application</h1> </center>

In [ ]:
#Importation of variAous Libraries or packages...
import os
from os import path as op
# import geopandas as gdp
import geemap
import ee
import geemap.foliumap as geemap

# Making module or packages collection attribute became callable..
import collections
collections.Callable = collections.abc.Callable

#plotting packages
import pandas as pd#Used for data analysis also include DataFrame df data structure
import matplotlib.pyplot as plt#For plotting of dataframes
import numpy as np
# import altair as alt#declarative visualization(Used for charting)

import ipywidgets as widgets

from ipyleaflet import WidgetControl
from ipywidgets import HTML
from IPython.display import display, clear_output
from ipywidgets import AppLayout

from geemap import chart #for chart plotting


# #packages used in database connection.
# import pandas.io.sql as sqlio
# import psycopg2 as ps

#Main function is to out various warning that might arise such as new upadates in package version
import warnings
warnings.filterwarnings("ignore")

import folium

Map=geemap.Map()
Map.setCenter(-0.39347351802909686, 36.94615326745641)

from geemap import Map
from ipywidgets import Button, Layout

map_layout= Map(layout=Layout(height='100px'))
map_layout.setCenter(-58.5568977590647, 2.019136139838323,6)
 
import geemap.chart as chart


In [ ]:
#Setting various buttons for the final dsplay of the results and the output widgets or setting the intractive keys
style = {'description_width': 'initial'}
#General output widget
output_widget = widgets.Output(layout={'border': '1px solid red','align-items':'center','background-color':'red'})


#General output control tool.
output_control = WidgetControl(widget=output_widget, position='topright')
#Map output control
map_layout.add_control(output_control)

#BUTTON  and TEXT SECTION 
Enter_project = widgets.Text(
    placeholder='Enter a project name(if any)',
    description='project selection:',
    disabled=False,
)

#styling method that can be used-----'primary', 'success', 'info', 'warning', 'danger',)
#Data collection button: Selection of collected data from the field.

#AOI Area of Interest Selection of study area.
AOI = widgets.Button(
    description='Temperature',
    button_style='success',
    tooltip='Draw you boundaries',
    style=style
)


crop_health= widgets.Button(
    description='Wind speed',
    button_style='success',
    tooltip='Wind speed',
    style=style
)

Export_data = widgets.Button(
    description='Rainfall Data',
    button_style='success',
    tooltip='Download classified results',
    style=style
)
#Charts
Rainfall_charts = widgets.Button(
    description='Generate Rainfall Charts',
    button_style='warning',
    tooltip='Download classified results',
    style=style
)
Temp_charts = widgets.Button(
    description='Generate Temperature Charts',
    button_style='warning',
    tooltip='Generate Temperature Charts',
    style=style
)
wind_speed = widgets.Button(
    description=' Generate Wind_Speed Charts',
    button_style='warning',
    tooltip='Generate Wind_Speed Charts',
    style=style
)
reset = widgets.Button(
    description='Reset',
    button_style='danger',
    tooltip='Reset me',
    style=style
)



w = widgets.Dropdown(
    options=['--Select--','Andhra Pradesh','Arunachal Pradesh','Assam','Uttarakhand', 'Haryana', 'Gujarat', 'Uttar Pradesh', 'Punjab'],
    value='--Select--',
    description='Select State:',
)

In [ ]:
#Trial to avoid printing error in the code but a message to the user
def AOI_clicked(b):
        with output_widget:
            output_widget.clear_output()
           
            try:
                global boundaries
                boundaries=map_layout.user_rois.geometry()
                map_layout.addLayer(boundaries,{},"Region of Intrest")
            
            
            except Exception as e:
                print('Please select Your area of Interest.....')
            else:
                print("successfully drawn your Region of Interest ")
        
AOI.on_click(AOI_clicked)



In [ ]:
#Trial to avoid printing error in the code but a message to the user
#WindSpeed
def AOI_clicked(change):
        with output_widget:
            output_widget.clear_output()
            try:
                map_layout.remove_legends() 
                map_layout.remove_colorbars()
                global region
                region=map_layout.user_rois
                map_layout.addLayer(region,{},"Region of Intrest")
                global windspeed_Data
                windspeed_Data = ee.ImageCollection('IDAHO_EPSCOR/TERRACLIMATE') \
                  .filter(ee.Filter.date('2021-01-01', '2021-12-01'))\
                  .median()\
                   .clip(region)
                windspeed_Data = windspeed_Data.select('vs')
                windspeed_Data_Vis = {
  'min': 0.01,
  'max': 2923.0,
  'palette': [
    '1a3678', '2955bc', '5699ff', '8dbae9', 'acd1ff', 'caebff', 'e5f9ff',
    'fdffb4', 'ffe6a2', 'ffc969', 'ffa12d', 'ff7c1f', 'ca531a', 'ff0000',
    'ab0000'
  ],
}
                map_layout.addLayer(windspeed_Data, windspeed_Data_Vis, 'Wind Speed')
                
                vis_params = {
    'min': -20,
    'max': 79,
    'palette': [
    '1a3678', '2955bc', '5699ff', '8dbae9', 'acd1ff', 'caebff', 'e5f9ff',
    'fdffb4', 'ffe6a2', 'ffc969', 'ffa12d', 'ff7c1f', 'ca531a', 'ff0000',
    'ab0000'
  ]
}
                colors = vis_params['palette']
                vmin = vis_params['min']
                vmax = vis_params['max']

                map_layout.add_colorbar(vis_params,label=' Temperature  Readings')
            except Exception as e:
                print('Please select Your area of Interest.....')
            else:
                print("successfully drawn your Region of Interest")

crop_health.on_click(AOI_clicked)

In [ ]:
#Trial to avoid printing error in the code but a message to the user
#######Rainfall

def AOI_clicked(change):
        with output_widget:
            output_widget.clear_output()
           
            try:
                map_layout.remove_legends() 
                map_layout.remove_colorbars()
                global region
                region=map_layout.user_rois
                map_layout.addLayer(region,{},"Region of Intrest")
            
                dataset = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY') \
                  .filter(ee.Filter.date('2021-01-01', '2021-12-01'))\
                  .median()\
                   .clip(region)
                global precipitation
                precipitation = dataset.select('precipitation')
                precipitationVis = {
  'min': 1.0,
  'max': 17.0,
  'palette': ['001137', '0aab1e', 'e7eb05', 'ff4a2d', 'e90000'],
}
                vis_params = {
    'min': 0.01,
    'max': 2923,
    'palette': ['001137', '0aab1e', 'e7eb05', 'ff4a2d', 'e90000']
}
                colors = vis_params['palette']
                vmin = vis_params['min']
                vmax = vis_params['max']
                map_layout.addLayer(precipitation, precipitationVis, 'Precipitation')
                map_layout.add_colorbar(vis_params,label=' Rainfall Readings')
            except Exception as e:
                print('Please select Your area of Interest.....')
            else:
                print("successfully the selected algorithm for Rainfall")

Export_data.on_click(AOI_clicked)

In [ ]:
user_out = widgets.Output(layout={'border': '1px solid green'})


In [ ]:
#Trial to avoid printing error in the code but a message to the user
#######Temperture

def AOI_clicked(change):
        with user_out:
            output_widget.clear_output()
            try:
                global region
                region=map_layout.user_rois
                map_layout.addLayer(region,{},"Region of Intrest")
                my_sample = windspeed_Data.sample(region, 500)
                property = 'tmmx'
                options = {
    "title": 'TEMPERATURE READINGS',
    "xlabel": 'Time Frame',
    "ylabel": 'Maximum(°C) Temperature ',
    "colors": ['#1d6b99'],
}
                chart.feature_histogram(my_sample, property, **options)
            except Exception as e:
                print('Error generating Temperature charts ...Please select a smaller region')
            else:
                print("successfully the selected algorithm for temperature ")
Temp_charts.on_click(AOI_clicked)

In [ ]:
#Trial to avoid printing error in the code but a message to the user
#######windspeed

def AOI_clicked(change):
        with user_out:
            output_widget.clear_output()
            try:
                global region
                
                region=map_layout.user_rois
                map_layout.addLayer(region,{},"Region of Intrest")
                my_sample = windspeed_Data.sample(region, 500)
                property = 'vs'
                options = {
    "title": 'Wind Speed',
    "xlabel": 'Time Frame',
    "ylabel": 'Wind Speed(mm)',
    "colors": ['#1d6b99'],
}
                chart.feature_histogram(my_sample, property, **options)
            except Exception as e:
                print('Error generating wind speed charts...Please select a smaller region')
            else:
                print("successfully the selected algorithm for wind speed")
wind_speed.on_click(AOI_clicked)

In [ ]:
#Trial to avoid printing error in the code but a message to the user
#######Temperture

def AOI_clicked(change):
        with user_out:
            output_widget.clear_output()
            try:
                global region
                region=map_layout.user_rois
                map_layout.addLayer(region,{},"Region of Intrest")
                region=map_layout.user_rois
                map_layout.addLayer(region,{},"Region of Intrest")
                my_sample = precipitation.sample(region, 500)
                property = 'precipitation'

                options = {
    "title": 'Rainfall',
    "xlabel": 'Time Frame',
    "ylabel": 'precipitation(mm)',
    "colors": ['#1d6b99'],
}
                chart.feature_histogram(my_sample, property, **options)
            except Exception as e:
                print('Error generating Rainfall charts...Please select a smaller region')
            else:
                print("successfully the selected algorithm for Rainfall")

Rainfall_charts.on_click(AOI_clicked)

In [ ]:
def reset_me(b):
    with output_widget:
        output_widget.clear_output()
        try:
            map_layout.remove_legends() #Remove legends(Classification, Vegetation Indices legends)
            map_layout.remove_colorbars()#Removal of various colour bars ranging from NDVI to EVI
            map_layout.remove_drawn_features()#Removal of drawn features on the map layout eg circle or polygon features.
            #Clear or removal of layers based on the name used:
            map_layout.remove_ee_layer("Precipitation")#Removes D-krops points
            map_layout.remove_ee_layer("Region of Intrest ")#Removes AOI i.e shapefile.
            map_layout.remove_ee_layer('Maximum Temperature')#Removes preprocess sentine 2A image.
            map_layout.remove_ee_layer( 'Wind Speed')#Removes classification results
         
           
        except Exception as e:
            print("You may have restart all available features or an error occurred!!!")
        else:
            print("Your cleared successfully... please start again!!!")
reset.on_click(reset_me)  


<h3>CHIRPS Background</h3><br>

Since 1999, USGS and CHC scientists—supported by funding from USAID, NASA, and NOAA—have developed techniques for producing rainfall maps, especially in areas where surface data is sparse.

Estimating rainfall variations in space and time is a key aspect of drought early warning and environmental monitoring. An evolving drier-than-normal season must be placed in a historical context so that the severity of rainfall deficits can be quickly evaluated. However, estimates derived from satellite data provide areal averages that suffer from biases due to complex terrain, which often underestimate the intensity of extreme precipitation events. Conversely, precipitation grids produced from station data suffer in more rural regions where there are less rain-gauge stations. CHIRPS was created in collaboration with scientists at the USGS Earth Resources Observation and Science (EROS) Center in order to deliver complete, reliable, up-to-date data sets for a number of early warning objectives, like trend analysis and seasonal drought monitoring.<br>
<h3>CHIRPS Rainfall</h3><br>
Climate Hazards Group InfraRed Precipitation with Station data (CHIRPS) is a 30+ year quasi-global rainfall dataset. CHIRPS incorporates 0.05° resolution satellite imagery with in-situ station data to create gridded rainfall time series for trend analysis and seasonal drought monitoring.<br>

<h3>How the model was trained</h3><br>
The above results were obtained by the capability for one to drawn any region at any point, then extracting the drawn pixels and exporting it as a csv file to a cloud storage.The extracted csv file will be fetched and feed into the charts and graphs for it to be visualize.<br>
<center><h5>Powered by Geemap and ipywidgets</h5></center><br>
<center><h5>Author:@ Mosong Geospatial Solutions.. </h5></center><br>
<center><img src="mylogo.png" style="width: 80px;border-radius: 60%;
"></center>

<!-- <center><img src="mylogo.png" style="width: 80px;border-radius: 60%;
"></center> -->